In [36]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from config import g_key


In [37]:
df=pd.read_csv("final_311_census.csv")
df.head()

,CREATED_ON,CREATE_DATE,REQUEST_TYPE,REQUEST_ORIGIN,DEPARTMENT,NEIGHBORHOOD,X,Y,location,ZIP,Household Income,Population,Median Age,Per Capita Income,Poverty Count
0,2015-04-20T08:38:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.015178,40.392419,"40.3924186,-80.01517829999999",15226,54441,13632,39.5,29870,1329
1,2015-04-20T08:42:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.018696,40.400007,"40.4000074,-80.0186958",15226,54441,13632,39.5,29870,1329
2,2015-04-20T08:43:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.016089,40.399618,"40.399617799999994,-80.01608940000001",15226,54441,13632,39.5,29870,1329
3,2015-04-20T09:15:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.007267,40.383212,"40.3832124,-80.0072673",15226,54441,13632,39.5,29870,1329
4,2015-04-20T09:16:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.009981,40.384611,"40.3846107,-80.0099811",15226,54441,13632,39.5,29870,1329


In [38]:
df['month'] = pd.DatetimeIndex(df['CREATE_DATE']).month
df.head()

,CREATED_ON,CREATE_DATE,REQUEST_TYPE,REQUEST_ORIGIN,DEPARTMENT,NEIGHBORHOOD,X,Y,location,ZIP,Household Income,Population,Median Age,Per Capita Income,Poverty Count,month
0,2015-04-20T08:38:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.015178,40.392419,"40.3924186,-80.01517829999999",15226,54441,13632,39.5,29870,1329,4
1,2015-04-20T08:42:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.018696,40.400007,"40.4000074,-80.0186958",15226,54441,13632,39.5,29870,1329,4
2,2015-04-20T08:43:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.016089,40.399618,"40.399617799999994,-80.01608940000001",15226,54441,13632,39.5,29870,1329,4
3,2015-04-20T09:15:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.007267,40.383212,"40.3832124,-80.0072673",15226,54441,13632,39.5,29870,1329,4
4,2015-04-20T09:16:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.009981,40.384611,"40.3846107,-80.0099811",15226,54441,13632,39.5,29870,1329,4


In [39]:
bins = [0, 30000, 60000, 100000, 500000]

# Create the names for the four bins
group_names = ["<30K", "30 to 60K", "60 to 100K", ">100K"]

In [40]:
df["Income Category"] = pd.cut(df["Household Income"], bins, labels=group_names)
df

,CREATED_ON,CREATE_DATE,REQUEST_TYPE,REQUEST_ORIGIN,DEPARTMENT,NEIGHBORHOOD,X,Y,location,ZIP,Household Income,Population,Median Age,Per Capita Income,Poverty Count,month,Income Category
0,2015-04-20T08:38:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.015178,40.392419,"40.3924186,-80.01517829999999",15226,54441,13632,39.5,29870,1329,4,30 to 60K
1,2015-04-20T08:42:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.018696,40.400007,"40.4000074,-80.0186958",15226,54441,13632,39.5,29870,1329,4,30 to 60K
2,2015-04-20T08:43:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.016089,40.399618,"40.399617799999994,-80.01608940000001",15226,54441,13632,39.5,29870,1329,4,30 to 60K
3,2015-04-20T09:15:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.007267,40.383212,"40.3832124,-80.0072673",15226,54441,13632,39.5,29870,1329,4,30 to 60K
4,2015-04-20T09:16:00,2015-04,Potholes,Call Center,DPW - Street Maintenance,Brookline,-80.009981,40.384611,"40.3846107,-80.0099811",15226,54441,13632,39.5,29870,1329,4,30 to 60K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22408,2018-01-30T16:28:00,2018-01,Potholes,Call Center,DPW - Street Maintenance,Squirrel Hill South,-79.925185,40.423677,"40.42367720000001,-79.9251852",15207,44894,12075,40.1,25058,2221,1,30 to 60K
22409,2018-02-02T15:36:00,2018-02,Potholes,Call Center,DPW - Street Maintenance,Squirrel Hill South,-79.925185,40.423677,"40.42367720000001,-79.9251852",15207,44894,12075,40.1,25058,2221,2,30 to 60K
22410,2019-02-11T09:14:00,2019-02,Snow/Ice removal,Call Center,DPW - Street Maintenance,Squirrel Hill South,-79.949334,40.428703,"40.42870294,-79.94933354",15207,44894,12075,40.1,25058,2221,2,30 to 60K
22411,2018-02-23T16:03:00,2018-02,Potholes,Call Center,DPW - Street Maintenance,South Side Slopes,-79.975504,40.424848,"40.42484843,-79.97550428",15209,55248,12225,47.0,33908,1456,2,30 to 60K


In [41]:
df = df.groupby("Income Category")
df.()

AttributeError: 'DataFrameGroupBy' object has no attribute 'values_count'